In [1]:
import datetime
import pandas as pd
import pprint
import pyaurorax

aurorax = pyaurorax.PyAuroraX()

# Search for conjunctions

The cornerstone aspect of the AuroraX search engine is the ability to automatically find conjunctions between ground-based instruments and spacecrafts. One aspect that makes AuroraX's conjunction search different from other solutions is that it will only find times where there is ground-based data available. It can also limit results to conjunctions where certain spacecraft instruments were operating. This integration with data availability saves us all valuable minutes and hours when surveying or searching for data that could be helpful for your next paper. 

A great way to get introduced to the AuroraX Conjunction Search capabilities is to head on over the [web page](https://aurorax.space/conjunctionSearch/standard) and play around with UI there. Any request you can do in the web interface, you can also do using the below function.

An common stumbling block for making search queries is being unclear on the values that you can use for the program, platform, instrument type, etc. The AuroraX search engine is underpinned by 'data sources', and this is where the information can be found. Use the `aurorax.search.sources.list()` function to show all the available data sources that you can use when constructing search queries. For metadata filter requests, the information is also contained in the data sources that identify the available filter keys and values. We'll have a closer look at this in the metadata filter examples further below.

In [2]:
# The data sources are what we use for search queries. We list some below,
# and in the following search queries in this notebook, we utilize this
# information for the program, platform, instrument type fields.

# let's list the first 10 data sources just to get us a table view of a few
aurorax.search.sources.list_in_table(limit=20)

# the below line gets all data sources, which we'll use later to explore the
# available metadata filters
sources = aurorax.search.sources.list()

Identifier   Program      Platform       Instrument Type    Source Type   Display Name   
3            swarm        swarma         footprint          leo           Swarm A        
29           swarm        swarmb         footprint          leo           Swarm B        
30           swarm        swarmc         footprint          leo           Swarm C        
32           epop         epop           footprint          leo           ePOP           
33           themis       themisa        footprint          heo           THEMIS-A       
34           themis       themisb        footprint          heo           THEMIS-B       
35           themis       themisc        footprint          heo           THEMIS-C       
36           themis       themisd        footprint          heo           THEMIS-D       
37           themis       themise        footprint          heo           THEMIS-E       
38           arase        arase          footprint          heo           Arase          
39        

Now that we know a bit more about how the data sources come into play with the search engine, let's do a simple conjunction search.

In [3]:
# search for conjunctions between any THEMIS ASI and any Swarm spacecraft
#
# set timeframe and distance
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 1, 6, 59, 59)
distance = 500

# set criteria blocks
ground = [aurorax.search.GroundCriteriaBlock(programs=["themis-asi"])]
space = [aurorax.search.SpaceCriteriaBlock(programs=["swarm"])]

# perform search
s = aurorax.search.conjunctions.search(start, end, distance, ground=ground, space=space, verbose=True)

[2025-01-26 15:32:48.769932] Search object created
[2025-01-26 15:32:48.792916] Request submitted
[2025-01-26 15:32:48.792981] Request ID: 9ae39ce6-9ebf-4cb0-bd1a-09e6fa8d93c5
[2025-01-26 15:32:48.793009] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/9ae39ce6-9ebf-4cb0-bd1a-09e6fa8d93c5
[2025-01-26 15:32:48.793033] Waiting for data ...
[2025-01-26 15:32:50.212110] Checking for data ...
[2025-01-26 15:32:50.630934] Data is now available
[2025-01-26 15:32:50.631127] Retrieving data ...
[2025-01-26 15:32:51.116989] Retrieved 376.7 kB of data containing 49 records


In [4]:
# output data
#
# NOTE: while here we format the results into a Pandas dataframe, this
# is not required. We actually don't include Pandas as a dependency since
# it's used simply as a nice add-on to view data. If you're good with slicing
# and dicing lists and dictionaries, you'll be fine without it.
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)

# let's show the first 10 conjunctions, sorted by 'start' time
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
43,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",206.588207,260.819127,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
15,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",192.029390,336.631805,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
11,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",282.877438,285.744881,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
38,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",195.554240,298.419001,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
30,nbtrace,2020-01-01T00:11:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",53.391843,441.814398,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:11:00
4,nbtrace,2020-01-01T00:12:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",89.436436,332.998863,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:13:00
17,nbtrace,2020-01-01T00:13:00,2020-01-01T00:13:00,"[DataSource(identifier=54, program='themis-asi...",401.883399,401.883399,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:13:00,2020-01-01T00:13:00
9,nbtrace,2020-01-01T00:13:00,2020-01-01T00:15:00,"[DataSource(identifier=50, program='themis-asi...",105.345328,488.349867,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:15:00
37,nbtrace,2020-01-01T00:13:00,2020-01-01T00:14:00,"[DataSource(identifier=50, program='themis-asi...",215.337413,330.343161,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:13:00
7,nbtrace,2020-01-01T00:44:00,2020-01-01T00:46:00,"[DataSource(identifier=48, program='themis-asi...",114.890751,488.203200,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:45:00,2020-01-01T00:46:00


# Search for conjunctions with metadata filters

Using the metadata filters to help search for conjunctions is one of the more advanced, and highly powerful, tools available. There are many metadata filters for spacecrafts, and some very useful ML-derived filters for ground-based all-sky imagers. 

An important part of being able to utilize the metadata filters is knowing the available keys and values. Each data source record has an attribute named `ephemeris_metadata_schema`. 

In [5]:
# using the data source listing that we retrieved further above, let's
# have a look at one of the records
#
# for the first data source, print only the first metadata filter info
print(sources[0].program, sources[0].platform, sources[0].instrument_type)
pprint.pprint(sources[0].ephemeris_metadata_schema[0])  # type: ignore

swarm swarma footprint
{'allowed_values': ['north polar cap',
                    'north cusp',
                    'north cleft',
                    'north auroral oval',
                    'north mid-latitude',
                    'low latitude'],
 'data_type': 'string',
 'description': 'Region based on where the magnetic field line that passes '
                "through the spacecraft intersects the Earth's surface in the "
                "Earth's northern magnetic hemisphere",
 'field_name': 'nbtrace_region',
 'searchable': True}


We see above, just one of the metadata filters we can use for the Swarm-A spacecraft. We'll leave it up to you from here to explore the additional filters for Swarm, and the available filters for any other data source. 

If you prefer to look at all the available metadata filters in a web browser instead, you can head on over to the [AuroraX Conjunction Search webpage](https://aurorax.space/conjunctionSearch/standard). Select your data source(s), and click on the '+' icon for metadata filters, and a modal will pop up. All metadata filters for the selected data sources are displayed in the modal.

Now that we understand the metadata filter keys and values a bit more, let's dive into some examples. We'll start with a simple example below, where we search for conjunctions filtering for when spacecrafts are in the north polar cap. The regions available to choose from are all directly pulled from SSCWeb. Almost all data used by AuroraX for spacecrafts is from SSCWeb without any alterations, only organization in the AuroraX database to enable the search engine to function.

If you're interested in interacting with the ML-derived metadata for ground-based instruments, have a look at [this crib sheet](https://github.com/aurorax-space/pyaurorax/tree/main/examples/notebooks/search/search_ephemeris_ml.ipynb).

In [6]:
# search for conjunctions between any THEMIS-ASI or REGO instrument, and any Swarm
# instrument where the north B-trace region is 'north polar cap'.
#
# NOTE: this region metadata is not derived by AuroraX, but instead directly from
# SSCWeb. See the above text for more info.
#
# set timeframe and distance
start = datetime.datetime(2019, 2, 1, 0, 0, 0)
end = datetime.datetime(2019, 2, 10, 23, 59, 59)
distance = 500

# set ground criteria block
ground = [aurorax.search.GroundCriteriaBlock(programs=["themis-asi", "rego"])]

# set space criteria block, with a metadata filter
expression1 = aurorax.search.MetadataFilterExpression(key="nbtrace_region", values="north polar cap", operator="=")
metadata_filter = aurorax.search.MetadataFilter(expressions=[expression1])
space = [aurorax.search.SpaceCriteriaBlock(programs=["swarm"], metadata_filters=metadata_filter)]

# perform search
s = aurorax.search.conjunctions.search(start, end, distance, ground=ground, space=space, verbose=True)

[2025-01-26 15:32:51.227343] Search object created
[2025-01-26 15:32:51.273720] Request submitted
[2025-01-26 15:32:51.273786] Request ID: 88207075-a7de-445d-a9d3-8b05bb5a74f9
[2025-01-26 15:32:51.273839] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/88207075-a7de-445d-a9d3-8b05bb5a74f9
[2025-01-26 15:32:51.273863] Waiting for data ...
[2025-01-26 15:32:52.693023] Checking for data ...
[2025-01-26 15:32:53.107764] Data is now available
[2025-01-26 15:32:53.108018] Retrieving data ...
[2025-01-26 15:32:53.752628] Retrieved 2.1 MB of data containing 273 records


In [7]:
# view the first 10 conjunctions
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
97,nbtrace,2019-02-01T02:20:00,2019-02-01T02:21:00,"[DataSource(identifier=53, program='themis-asi...",153.987890,371.020379,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:20:00,2019-02-01T02:21:00
197,nbtrace,2019-02-01T02:20:00,2019-02-01T02:21:00,"[DataSource(identifier=80, program='rego', pla...",153.987890,371.020379,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:20:00,2019-02-01T02:21:00
230,nbtrace,2019-02-01T02:21:00,2019-02-01T02:22:00,"[DataSource(identifier=83, program='rego', pla...",267.685472,300.984658,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:22:00,2019-02-01T02:21:00
170,nbtrace,2019-02-01T03:55:00,2019-02-01T03:56:00,"[DataSource(identifier=78, program='rego', pla...",471.096543,490.131048,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T03:56:00,2019-02-01T03:55:00
233,nbtrace,2019-02-01T03:57:00,2019-02-01T03:57:00,"[DataSource(identifier=83, program='rego', pla...",496.774661,496.774661,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T03:57:00,2019-02-01T03:57:00
151,nbtrace,2019-02-01T04:55:00,2019-02-01T04:56:00,"[DataSource(identifier=83, program='rego', pla...",424.997148,445.453544,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T04:55:00,2019-02-01T04:56:00
252,nbtrace,2019-02-01T04:55:00,2019-02-01T04:55:00,"[DataSource(identifier=83, program='rego', pla...",438.433977,438.433977,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T04:55:00,2019-02-01T04:55:00
173,nbtrace,2019-02-01T05:31:00,2019-02-01T05:31:00,"[DataSource(identifier=78, program='rego', pla...",440.820767,440.820767,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T05:31:00,2019-02-01T05:31:00
262,nbtrace,2019-02-01T06:28:00,2019-02-01T06:30:00,"[DataSource(identifier=83, program='rego', pla...",234.199376,484.697629,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T06:29:00,2019-02-01T06:28:00
155,nbtrace,2019-02-01T06:29:00,2019-02-01T06:30:00,"[DataSource(identifier=83, program='rego', pla...",280.074450,466.631937,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T06:29:00,2019-02-01T06:30:00


# Multiple ground and space instruments, and advanced distances

Since the `ground` and `space` paramters are lists, we can search for conjunctions between multiple ground and space instruments. Each item of the lists are evaluated as a logical AND statement when searching for conjunctions. In the [web interface](https://aurorax.space/conjunctionSearch/standard), these are called 'criteria blocks' and are represented as each row in the table used for setting up a query. We do the same naming convention in PyAuroraX.

You'll also notice that you can set distances between each list item (ground1-space1, ground1-space2, etc.). This can be quite powerful, but just remember to think about these values to ensure you're doing the right kind of distance evaluation between each. It can get confusing easily for each criteria block you add. A value of `None` will tell the search engine to not care about the distance between those two items.

In [8]:
# search for conjunctions between any REGO instrument, any TREx instrument,
# any Swarm spacecraft, and any THEMIS spacecraft, up to 500 km apart for
# each ground instrument and spacecraft.
#
# since we have four list items, we can refer to this as a 'quadruple conjunction'
# search
#
# set timeframe
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 4, 23, 59, 59)

# set criteria blocks
ground = [
    aurorax.search.GroundCriteriaBlock(programs=["rego"]),
    aurorax.search.GroundCriteriaBlock(programs=["trex"]),
]
space = [
    aurorax.search.SpaceCriteriaBlock(programs=["swarm"]),
    aurorax.search.SpaceCriteriaBlock(programs=["themis"]),
]

# set advanced distances
advanced_distances = {
    "ground1-ground2": None,
    "ground1-space1": 500,
    "ground1-space2": 500,
    "ground2-space1": 500,
    "ground2-space2": 500,
    "space1-space2": None
}

# perform search
s = aurorax.search.conjunctions.search(start, end, advanced_distances, ground=ground, space=space, verbose=True)

[2025-01-26 15:32:53.810371] Search object created
[2025-01-26 15:32:53.839788] Request submitted
[2025-01-26 15:32:53.839845] Request ID: fe457aff-dff7-4a12-adb8-be5a4ccc3b03
[2025-01-26 15:32:53.839862] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/fe457aff-dff7-4a12-adb8-be5a4ccc3b03
[2025-01-26 15:32:53.839875] Waiting for data ...
[2025-01-26 15:32:55.259225] Checking for data ...
[2025-01-26 15:32:56.725696] Checking for data ...
[2025-01-26 15:32:58.141265] Checking for data ...
[2025-01-26 15:32:59.564312] Checking for data ...
[2025-01-26 15:33:00.000148] Data is now available
[2025-01-26 15:33:00.000351] Retrieving data ...
[2025-01-26 15:33:00.473834] Retrieved 87.8 kB of data containing 8 records


In [9]:
# show the results
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
4,nbtrace,2020-01-02T13:00:00,2020-01-02T13:01:00,"[DataSource(identifier=81, program='rego', pla...",384.347496,440.461162,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-02T13:01:00,2020-01-02T13:00:00
7,nbtrace,2020-01-02T13:00:00,2020-01-02T13:01:00,"[DataSource(identifier=81, program='rego', pla...",384.347496,440.461162,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-02T13:01:00,2020-01-02T13:00:00
0,nbtrace,2020-01-03T12:33:00,2020-01-03T12:35:00,"[DataSource(identifier=81, program='rego', pla...",111.324353,485.848555,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:34:00,2020-01-03T12:35:00
1,nbtrace,2020-01-03T12:33:00,2020-01-03T12:35:00,"[DataSource(identifier=81, program='rego', pla...",111.324353,485.848555,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:34:00,2020-01-03T12:35:00
3,nbtrace,2020-01-03T12:33:00,2020-01-03T12:34:00,"[DataSource(identifier=81, program='rego', pla...",122.118464,294.065540,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:34:00,2020-01-03T12:33:00
6,nbtrace,2020-01-03T12:33:00,2020-01-03T12:34:00,"[DataSource(identifier=81, program='rego', pla...",122.118464,294.065540,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:34:00,2020-01-03T12:33:00
2,nbtrace,2020-01-03T12:40:00,2020-01-03T12:42:00,"[DataSource(identifier=81, program='rego', pla...",255.883772,476.507384,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:41:00,2020-01-03T12:40:00
5,nbtrace,2020-01-03T12:40:00,2020-01-03T12:42:00,"[DataSource(identifier=81, program='rego', pla...",255.883772,476.507384,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:41:00,2020-01-03T12:40:00


There is a helper function available to assist with the advanced distances. It can be easy to miss a pairing when writing the dictionary manually, so we can use the `aurorax.search.conjunctions.create_advanced_distances_dict()` function to create the pairings dictionary for us. After we have it, we can go in an adjust the values as we desire.

Let's do the exact same example as above, but instead we'll utilize this helper function to show you how to use it.

In [10]:
# set timeframe
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 4, 23, 59, 59)

# set criteria blocks
ground = [
    aurorax.search.GroundCriteriaBlock(programs=["rego"]),
    aurorax.search.GroundCriteriaBlock(programs=["trex"]),
]
space = [
    aurorax.search.SpaceCriteriaBlock(programs=["swarm"]),
    aurorax.search.SpaceCriteriaBlock(programs=["themis"]),
]

# set advanced distances
#
# NOTE: this time, we use the helper function and then adjust the values
# fpr between the two ground and two space criteria blocks to be None (since
# we don't care about those combos)
advanced_distances = aurorax.search.conjunctions.create_advanced_distance_combos(500, ground=2, space=2)
advanced_distances["ground1-ground2"] = None
advanced_distances["space1-space2"] = None

# perform search
s = aurorax.search.conjunctions.search(start, end, advanced_distances, ground=ground, space=space, verbose=True)

[2025-01-26 15:33:00.559201] Search object created
[2025-01-26 15:33:00.592819] Request submitted
[2025-01-26 15:33:00.592878] Request ID: d57134f3-ab89-41e9-9b9d-929277eddab5
[2025-01-26 15:33:00.592899] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/d57134f3-ab89-41e9-9b9d-929277eddab5
[2025-01-26 15:33:00.592918] Waiting for data ...
[2025-01-26 15:33:02.012461] Checking for data ...
[2025-01-26 15:33:03.445164] Checking for data ...
[2025-01-26 15:33:04.876161] Checking for data ...
[2025-01-26 15:33:06.306847] Checking for data ...
[2025-01-26 15:33:06.734596] Data is now available
[2025-01-26 15:33:06.734817] Retrieving data ...
[2025-01-26 15:33:07.210825] Retrieved 87.8 kB of data containing 8 records


# Search for conjunctions between spacecrafts only

Since the conjunction search engine boils down to just finding matches between any two data sources, we can easily extend this to find conjunctions between only spacecrafts.

In [11]:
# search for conjunctions between Swarm A or Swarm B, and any THEMIS
# spacecraft with the south B-trace region matching 'south polar cap'
#
# set timeframe and distance
start = datetime.datetime(2019, 1, 1, 0, 0, 0)
end = datetime.datetime(2019, 1, 1, 23, 59, 59)
distance = 500

# set space criteria blocks
space = [
    aurorax.search.SpaceCriteriaBlock(programs=["themis"]),
    aurorax.search.SpaceCriteriaBlock(programs=["swarm"],
                                      platforms=["swarma", "swarmb"],
                                      hemisphere=["southern"],
                                      metadata_filters=aurorax.search.MetadataFilter(expressions=[
                                          aurorax.search.MetadataFilterExpression("sbtrace_region", ["south polar cap"], operator="in"),
                                      ]))
]

# perform search
s = aurorax.search.conjunctions.search(start, end, distance, space=space, verbose=True)

[2025-01-26 15:33:07.229435] Search object created
[2025-01-26 15:33:07.253128] Request submitted
[2025-01-26 15:33:07.253168] Request ID: d190b5c3-8f0e-4231-b0fc-dfa46294a8ef
[2025-01-26 15:33:07.253183] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/d190b5c3-8f0e-4231-b0fc-dfa46294a8ef
[2025-01-26 15:33:07.253195] Waiting for data ...
[2025-01-26 15:33:08.673810] Checking for data ...
[2025-01-26 15:33:09.098682] Data is now available
[2025-01-26 15:33:09.098769] Retrieving data ...
[2025-01-26 15:33:09.538741] Retrieved 17.3 kB of data containing 2 records


In [12]:
# show the results
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
1,nbtrace,2019-01-01T10:01:00,2019-01-01T10:01:00,"[DataSource(identifier=33, program='themis', p...",311.183877,311.183877,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-01-01T10:01:00,2019-01-01T10:01:00
0,nbtrace,2019-01-01T12:08:00,2019-01-01T12:08:00,"[DataSource(identifier=33, program='themis', p...",215.817544,215.817544,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-01-01T12:08:00,2019-01-01T12:08:00


# Search for conjunctions with curated event lists

There exist several curated event lists in the AuroraX search engine, including Toshi Nishimura's substorm list ([DOI](https://doi.org/10.1002/2016JA022801)), Megan Gillies' field line resonance list ([DOI](https://doi.org/10.1007/s11214-021-00830-x)), Bea Gallardo-Lacourt's STEVE list ([DOI](https://doi.org/10.1029/2018JA025368)), and Makenzie Ratzlaff's torches and omega band lists. To quickly look through these event lists, you can navigate to [Swarm-Aurora](https://swarm-aurora.com/conjunctionFinder/) and use the 'custom import' dropdown to load the list you're interested in.

We can use the AuroraX search engine to find conjunctions with any of these lists. Let's have a look at an example of using Toshi's substorm list to find conjunctions with any THEMIS spacecrafts.

In [13]:
# we'll do a search between Toshi's substorm list and any THEMIS spacecraft
# over the year of 2008. We limit the year, since multi years will take a little
# longer to run than we'd like in a quick example like this notebook is for. Feel
# free to do it though!
#
# set timeframe and distance
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 12, 31, 23, 59, 59)
distance = 500

# set criteria blocks
space = [aurorax.search.SpaceCriteriaBlock(programs=["themis"])]
events = [aurorax.search.EventsCriteriaBlock(instrument_types=["substorm onset"])]

# do search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    space=space,
    events=events,
    verbose=True,
)

[2025-01-26 15:33:09.743072] Search object created
[2025-01-26 15:33:09.794376] Request submitted
[2025-01-26 15:33:09.794449] Request ID: a3de1b28-5af1-4f60-95b5-80565acb4eea
[2025-01-26 15:33:09.794458] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/a3de1b28-5af1-4f60-95b5-80565acb4eea
[2025-01-26 15:33:09.794465] Waiting for data ...
[2025-01-26 15:33:11.254974] Checking for data ...
[2025-01-26 15:33:12.699934] Checking for data ...
[2025-01-26 15:33:14.153576] Checking for data ...
[2025-01-26 15:33:14.577272] Data is now available
[2025-01-26 15:33:14.577448] Retrieving data ...
[2025-01-26 15:33:15.155150] Retrieved 885.5 kB of data containing 189 records


# Search for conjunctions with custom locations

We can also search for conjunctions with arbitrary geographic locations. Let's say you're planning to put an instrument somewhere and you want to see the conjunctions the new instrument will get with the Swarm spacecrafts. We can use the `custom_locations` parameter to perform the search.

Let's have a look at an example of finding conjunctions between Calgary and any Swarm spacecraft over the course of a day.

In [14]:
# find conjunctions between Calgary and any Swarm spacecraft over
# the course of a day
#
# set timeframe and distance
start = datetime.datetime(2018, 1, 1, 0, 0, 0)
end = datetime.datetime(2018, 1, 1, 23, 59, 59)
distance = 500

# set criteria blocks
space = [aurorax.search.SpaceCriteriaBlock(programs=["swarm"])]
custom = [aurorax.search.CustomLocationsCriteriaBlock(locations=[(51.05, -114.07)])]

# do search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    space=space,
    custom_locations=custom,
    verbose=True,
)

[2025-01-26 15:33:15.171189] Search object created
[2025-01-26 15:33:15.202415] Request submitted
[2025-01-26 15:33:15.202484] Request ID: ac72726d-0d58-4f8f-83ec-828238fffbd9
[2025-01-26 15:33:15.202511] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/ac72726d-0d58-4f8f-83ec-828238fffbd9
[2025-01-26 15:33:15.202535] Waiting for data ...
[2025-01-26 15:33:16.627834] Checking for data ...
[2025-01-26 15:33:17.053532] Data is now available
[2025-01-26 15:33:17.053735] Retrieving data ...
[2025-01-26 15:33:17.514883] Retrieved 58.3 kB of data containing 9 records


# Adjusting the conjunction types parameter

When performing conjunction searches, the parameter `conjunction_types` specifies the types of location data to use when finding conjunctions. The choices are: `nbtrace`, `sbtrace`, and `geographic`. 

If 'nbtrace' is set, then conjunctions will be found using the north B-field trace location data (northern magnetic footprints). If 'sbtrace' is set, then conjunctions will be found using the south B-field trace location data (southern magnetic footprints). And lastly, if 'geographic' is set, then conjunctions will be found using the geographic location data. By default, the conjunction types parameter is only 'nbtrace'. 

Let's do two searches that show using the conjunction types paramter several different ways.

In [15]:
# find conjunctions using both nbtrace and sbtrace for THEMIS-ASI and
# THEMIS spacecrafts
#
# set timeframe and distance
start = datetime.datetime(2018, 1, 1, 0, 0, 0)
end = datetime.datetime(2018, 1, 1, 23, 59, 59)
distance = 400

# set criteria blocks
ground = [aurorax.search.GroundCriteriaBlock(programs=["themis-asi"])]
space = [aurorax.search.SpaceCriteriaBlock(programs=["themis"])]

# do search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground,
    space=space,
    conjunction_types=["nbtrace", "sbtrace"],
    verbose=True,
)

[2025-01-26 15:33:17.535562] Search object created
[2025-01-26 15:33:17.578520] Request submitted
[2025-01-26 15:33:17.578588] Request ID: f877ec55-4024-485c-9c80-b27d3a0fd394
[2025-01-26 15:33:17.578616] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/f877ec55-4024-485c-9c80-b27d3a0fd394
[2025-01-26 15:33:17.578639] Waiting for data ...
[2025-01-26 15:33:18.998374] Checking for data ...
[2025-01-26 15:33:19.419461] Data is now available
[2025-01-26 15:33:19.419721] Retrieving data ...
[2025-01-26 15:33:19.894392] Retrieved 129.9 kB of data containing 22 records


In [16]:
# find conjunctions between an arbitrary location and any Swarm spacecraft's
# geographic location
#
# set timeframe and distance
start = datetime.datetime(2018, 1, 1, 0, 0, 0)
end = datetime.datetime(2018, 1, 4, 23, 59, 59)
distance = 200

# set criteria blocks
space = [aurorax.search.SpaceCriteriaBlock(programs=["swarm"])]
custom = [aurorax.search.CustomLocationsCriteriaBlock(locations=[(51.05, -114.07)])]

# do search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    space=space,
    custom_locations=custom,
    conjunction_types=["geographic"],
    verbose=True,
)

[2025-01-26 15:33:19.915929] Search object created
[2025-01-26 15:33:19.966158] Request submitted
[2025-01-26 15:33:19.966230] Request ID: 64d3e672-7527-4e22-a601-d1fce39c347e
[2025-01-26 15:33:19.966257] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/64d3e672-7527-4e22-a601-d1fce39c347e
[2025-01-26 15:33:19.966281] Waiting for data ...
[2025-01-26 15:33:21.383002] Checking for data ...
[2025-01-26 15:33:21.805975] Data is now available
[2025-01-26 15:33:21.806180] Retrieving data ...
[2025-01-26 15:33:22.264496] Retrieved 32.1 kB of data containing 5 records


# Explore results in Swarm-Aurora

Just like the web interface, you can also explore the results of a conjunction search in Swarm-Aurora. This is enabled by knowing the 'search request ID' value, and a handy helper function in PyAuroraX. The function will spawn a browser window and open up Swarm-Aurora, loading up the conjunction results that were found for that search request.

In [17]:
# we'll do a search between a couple THEMIS ASIs and any Swarm spacecrafts
start = datetime.datetime(2019, 1, 1, 0, 0, 0)
end = datetime.datetime(2019, 1, 3, 23, 59, 59)
ground = [aurorax.search.GroundCriteriaBlock(programs=["themis-asi"], platforms=["fort smith", "gillam"])]
space = [aurorax.search.SpaceCriteriaBlock(programs=["swarm"], hemisphere=["northern"])]
distance = 500
s = aurorax.search.conjunctions.search(start=start, end=end, distance=distance, ground=ground, space=space, verbose=True)

[2025-01-26 15:33:22.288542] Search object created
[2025-01-26 15:33:22.313652] Request submitted
[2025-01-26 15:33:22.313718] Request ID: a444481c-c83f-4a20-a92b-df698bb05886
[2025-01-26 15:33:22.313745] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/a444481c-c83f-4a20-a92b-df698bb05886
[2025-01-26 15:33:22.313768] Waiting for data ...
[2025-01-26 15:33:23.733408] Checking for data ...
[2025-01-26 15:33:24.156067] Data is now available
[2025-01-26 15:33:24.156263] Retrieving data ...
[2025-01-26 15:33:24.625389] Retrieved 84.6 kB of data containing 11 records


With the search now done and saved to the `s` variable, we can use that object with a few helper functions.

NOTE: some IDEs can sometimes have issues figuring out the browser to use when opening Swarm-Aurora. To help with this, you can manually specify the 'browser' parameter to the helper function with one of the available values specified in the official `webbrowser` [module documentation](https://docs.python.org/3/library/webbrowser.html#webbrowser.get).

In [18]:
# open browser to Swarm-Aurora for this search
aurorax.search.conjunctions.swarmaurora.open_in_browser(s)

# if you want to force it to use Google Chrome, you can do the following
#
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s, browser="google-chrome")

# alternatively, you can also just get the web URL for Swarm-Aurora for this
# specific request, and copy paste it into the browser manually.
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=a444481c-c83f-4a20-a92b-df698bb05886


# Do the search step-by-step

Under the hood, the AuroraX API performs a conjunction search asynchronously. Note that this does not mean that it can be done using a Python async method; it means that PyAuroraX does more than just a single HTTP request when doing a search. With the API operating this way, it adds some more complexity within PyAuroraX but also opens the search up to some very important capabilities. 

The main capability enabled by this architecture is being able to perform queries for large timeframes, and/or between a large number of data sources. Queries like this can easily take several minutes, and cause browsers and programmatic HTTP requests to timeout.

Instead of using the `aurorax.search.conjunctions.search()` method, you can also perform a conjunction search step-by-step if you want more control over the process. One use case for this is if you want to start a series of conjunction searches, and then go through each getting the results back as they finish, as opposed to doing one search at a time.

In [19]:
# set up the search parameters
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 1, 6, 59, 59)
ground = [aurorax.search.GroundCriteriaBlock(programs=["themis-asi"])]
space = [aurorax.search.SpaceCriteriaBlock(programs=["swarm"])]
distance = 500

# create a Search object
s = pyaurorax.search.ConjunctionSearch(aurorax, start, end, distance, ground=ground, space=space)
s.pretty_print()

ConjunctionSearch:
  executed     : False
  completed    : False
  request_id   : 
  request      : None
  request_url  : 
  data_url     : 
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {}
  data         : 
  logs         : 


In [20]:
# submit the search to begin
s.execute()
s.pretty_print()

ConjunctionSearch:
  executed     : True
  completed    : False
  request_id   : 93713897-f744-45df-9add-bf8f9043a898
  request      : AuroraXAPIResponse [202] (Accepted)
  request_url  : https://api.aurorax.space/api/v1/conjunctions/requests/93713897-f744-45df-9add-bf8f9043a898
  data_url     : 
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {}
  data         : [0 conjunction results]
  logs         : [0 log messages]


In [21]:
# update the search request status
s.update_status()
s.pretty_print()

ConjunctionSearch:
  executed     : True
  completed    : False
  request_id   : 93713897-f744-45df-9add-bf8f9043a898
  request      : AuroraXAPIResponse [202] (Accepted)
  request_url  : https://api.aurorax.space/api/v1/conjunctions/requests/93713897-f744-45df-9add-bf8f9043a898
  data_url     : 
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {'search_request': {'request_id': '93713897-f744-45df-9add-bf8f9043a898', 'query...
  data         : [0 conjunction results]
  logs         : [5 log messages]


In [22]:
# wait for the data to be available
s.wait()
s.update_status()
s.pretty_print()

ConjunctionSearch:
  executed     : True
  completed    : True
  request_id   : 93713897-f744-45df-9add-bf8f9043a898
  request      : AuroraXAPIResponse [202] (Accepted)
  request_url  : https://api.aurorax.space/api/v1/conjunctions/requests/93713897-f744-45df-9add-bf8f9043a898
  data_url     : https://api.aurorax.space/api/v1/conjunctions/requests/93713897-f744-45df-9add-bf8f9043a898/data
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {'search_request': {'request_id': '93713897-f744-45df-9add-bf8f9043a898', 'query...
  data         : [0 conjunction results]
  logs         : [13 log messages]


In [23]:
# now that we know the request is complete, let's retrieve the data
s.get_data()
s.pretty_print()

ConjunctionSearch:
  executed     : True
  completed    : True
  request_id   : 93713897-f744-45df-9add-bf8f9043a898
  request      : AuroraXAPIResponse [202] (Accepted)
  request_url  : https://api.aurorax.space/api/v1/conjunctions/requests/93713897-f744-45df-9add-bf8f9043a898
  data_url     : https://api.aurorax.space/api/v1/conjunctions/requests/93713897-f744-45df-9add-bf8f9043a898/data
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {'search_request': {'request_id': '93713897-f744-45df-9add-bf8f9043a898', 'query...
  data         : [49 conjunction results]
  logs         : [13 log messages]


In [24]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
43,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",206.588207,260.819127,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
15,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",192.029390,336.631805,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
11,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",282.877438,285.744881,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
38,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",195.554240,298.419001,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
30,nbtrace,2020-01-01T00:11:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",53.391843,441.814398,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:11:00
4,nbtrace,2020-01-01T00:12:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",89.436436,332.998863,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:13:00
17,nbtrace,2020-01-01T00:13:00,2020-01-01T00:13:00,"[DataSource(identifier=54, program='themis-asi...",401.883399,401.883399,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:13:00,2020-01-01T00:13:00
9,nbtrace,2020-01-01T00:13:00,2020-01-01T00:15:00,"[DataSource(identifier=50, program='themis-asi...",105.345328,488.349867,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:15:00
37,nbtrace,2020-01-01T00:13:00,2020-01-01T00:14:00,"[DataSource(identifier=50, program='themis-asi...",215.337413,330.343161,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:13:00
7,nbtrace,2020-01-01T00:44:00,2020-01-01T00:46:00,"[DataSource(identifier=48, program='themis-asi...",114.890751,488.203200,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:45:00,2020-01-01T00:46:00
